In [1]:
class TimeDomainFilter:
    def __init__(self, K, period):
        self.K = K
        self.period = period
        self.alpha = (cos(np.degrees(self.K*360/self.period)) + sin(np.degrees(self.K*360/self.period)) - 1) / cos(np.degrees(self.K*360/self.period))
        print(self.alpha)
        #self.alpha = 0.2
        self.filter_bank = {
            'LPF-2pole' : [self.alpha**2, 0, 0, 1, -2*(1-self.alpha), (1-self.alpha)**2],
            'HPF-2pole' : [(1-self.alpha/2)**2, -2*(1-self.alpha/2)**2, (1-self.alpha/2)**2, 1, -2*(1-self.alpha), (1-self.alpha)**2],
            'HPF' : [1-self.alpha/2, -(1-self.alpha/2), 0, 1, -(1-self.alpha), 0],
            'decycler' : [self.alpha/2, self.alpha/2, 0, 1, -(1-self.alpha), 0]
        }
        
    def decycle(self, data, filter_type):
        # this is just a decycler right now.
        self.filter_constants = self.filter_bank[filter_type]
        b0,b1,b2 = self.filter_constants[0], self.filter_constants[1],self.filter_constants[2]
        a0,a1,a2 = self.filter_constants[3], self.filter_constants[4], self.filter_constants[5]
        
        outputs = np.copy(data)
        
        i = 1 # decycler is 1st order filter       
        while i < len(data):
            cur_output = b0 * data[i] + b1 * data[i-1] - a1 * outputs[i-1]
            outputs[i]=cur_output
            i += 1
        return outputs
    
    def high_pass_filter(self, data, filter_type):
        self.filter_constants = self.filter_bank[filter_type]
        b0,b1,b2 = self.filter_constants[0], self.filter_constants[1],self.filter_constants[2]
        a0,a1,a2 = self.filter_constants[3], self.filter_constants[4], self.filter_constants[5]
        
        outputs = np.copy(data)
        
        i = 1
        if filter_type == 'HPF-2pole':
            i = 2
            
        while i < len(data):
            cur_output = b0 * data[i] + b1 * data[i-1] + b2 * data[i-2] - a1 * outputs[i-1] - a2 * outputs[i-2]
            outputs[i]=cur_output
            i += 1
        return outputs

In [2]:
from math import cos, sin, pi
import numpy as np
K = 1 # test with only single pole for now
cutoff = 30